In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models 
from keras.layers import Conv2D, MaxPooling2D

class DataSource(object):
    def __init__(self):
        train = pd.read_csv('asgn2_data_train.csv')
        test = pd.read_csv('asgn2_data_20685600.csv')

        testX = np.array(test.drop(columns=['0']))
        trainX = np.array(train.drop(columns=['0']))

        testY = np.array(test.iloc[:,0])
        trainY = np.array(train.iloc[:,0])
        
        trainX = trainX.reshape(1580, 28, 28, 1)
        testX = testX.reshape(210, 28, 28, 1)
        self.trainX, self.trainY = trainX, trainY
        self.testX, self.testY = testX, testY

In [27]:
class CNN(object):
    def __init__(self):
        model = models.Sequential()
        model.add(layers.Conv2D(16, kernel_size = 2, activation = 'relu', input_shape =  (28,28,1)))
        model.add(layers.MaxPooling2D(strides = 2))
        model.add(layers.Conv2D(16, kernel_size = 2, activation = 'relu'))
        model.add(layers.MaxPooling2D(strides = 2))
        
        model.add(layers.Flatten())
        model.add(layers.Dense(120, activation = 'relu'))
        model.add(layers.Dense(64, activation = 'relu'))
        
        model.summary()
        self.model = model

In [31]:
class Train:
    def __init__(self):
        self.cnn = CNN()
        self.data = DataSource()
        
    def train(self):
        check_path = './ckpt/cp-{epoch:04d}.ckpt'
        save_model_cb = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only= True,verbose = 1, period = 10)
        
        self.cnn.model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
        self.cnn.model.fit(self.data.trainX, self.data.trainY, epochs = 10, callbacks = [save_model_cb])
        
        test_loss, test_acc = self.cnn.model.evaluate(self.data.testX,self.data.testX)
        print("accuracy: %.4f, tested %d images" % (test_acc, len(self.data.testY)))

In [37]:
test = Train()
test.train()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 27, 27, 16)        80        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 16)        1040      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 120)               69240     
_________________________________________________________________
dense_9 (Dense)              (None, 64)               

InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [32,64] and labels shape [25088]
	 [[node loss/dense_9_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /Users/evanaltshule/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_5849]

Function call stack:
distributed_function
